# Creating test data

In [ ]:
import numpy as np
from random import randint
from math import floor, fabs
from cplex.callbacks import UserCutCallback
import cplex  as CPX
import cplex.callbacks as CPX_CB
import sys
import time
import pandas as pd
from itertools import product
import random
from sklearn import preprocessing
import copy

# Parameters

In [ ]:
examples=['data/NewExample1','data/NewExample1Redundant0.1','data/NewExample1Redundant0.25','data/NewExample1Redundant0.5']
#The probability of having zero coefficient in linear combination for the test set:
ZeroProb=[0.0]

# Creating and saving the test data:

In [ ]:
for example in examples:
    for Z in ZeroProb:

        c = CPX.Cplex(example+'.lp')
        m=c.linear_constraints.get_num()
        n=c.variables.get_num()
        FinalCoeff=[]
        for i in range(m):
            my_row=[]
            for j in range(n):
                my_row.append(c.linear_constraints.get_coefficients(i,j))
            FinalCoeff.append(my_row) 
        FinalBounds=[]
        for i in range (n):
            tmp=[]
            for j in range (n):
                if i==j:
                    tmp.append(-1)
                else:
                    tmp.append(0)
            FinalBounds.append(tmp) 

        dfColumnName=[]  
        for i in range (n):
            dfColumnName.append("Var"+str(i))  
        dfColumnName.append("Output")            

        my_list = [Z] 
        another_list = [(1-Z)/100] *100
        my_list.extend(another_list) 

        TotalTime=0
        dfTest = np.empty((0,n+1))
        LenTestData=m+n/2

        for j in range(LenTestData):    
            testModel = CPX.Cplex(example+'.lp')
            obj= np.random.randint(-100, 100, n)
            for t in range(n):    
                testModel.objective.set_linear(t,obj[t])
            testModel.parameters.simplex.display.set(0)
            startTime=time.time()
            testModel.solve()
            TotalTime=TotalTime+time.time()-startTime
            normalized=obj/np.linalg.norm(obj)
            if testModel.solution.get_status()!=4:
                if testModel.solution.get_objective_value()>=0.00000000000001:
                    normalized = np.append(normalized,1)
                    dfTest=np.append(dfTest,np.array([normalized]), axis=0)
                else:
                    normalized = np.append(normalized,0)
                    dfTest=np.append(dfTest,np.array([normalized]), axis=0)
            else:
                normalized = np.append(normalized,1)
                dfTest=np.append(dfTest,np.array([normalized]), axis=0)

        for j in range(LenTestData):

            testModel = CPX.Cplex(example+'.lp')
            obj=np.zeros((1,n))
            for i in range (len(FinalCoeff)):
                alpha=np.random.choice(np.arange(0, 101), p=my_list)
                newrow=np.array(FinalCoeff[i])*alpha
                obj=np.add(obj,newrow)

            for t in range(n):    
                testModel.objective.set_linear(t,obj[0][t])
            testModel.parameters.simplex.display.set(0)
            startTime=time.time()
            testModel.solve()
            TotalTime=TotalTime+time.time()-startTime
            normalized=obj/np.linalg.norm(obj)
            if testModel.solution.get_status()!=4:
                if testModel.solution.get_objective_value()>=0.00000000000001:
                    normalized = np.append(normalized,1)
                    dfTest=np.append(dfTest,np.array([normalized]), axis=0)
                else:
                    normalized = np.append(normalized,0)
                    dfTest=np.append(dfTest,np.array([normalized]), axis=0)
            else:
                normalized = np.append(normalized,1)
                dfTest=np.append(dfTest,np.array([normalized]), axis=0)   
        Final=TotalTime/len(dfTest)
        dfFinalTest = pd.DataFrame(data=dfTest, columns=[dfColumnName])
        dfFinalTest.to_csv(r''+example+'test'+'.csv')     